In [1]:
#r "C:\BoSSS\experimental\internal\src\experimental\L4-application\InternalTestRunner\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
using ilPSP.Connectors.Matlab;

In [1]:
using System.Diagnostics;
using BoSSS.Solution.NSECommon;
using static System.Math;
using BoSSS.Solution.Queries;


In [1]:
using System.IO;

In [1]:
using BoSSS.Application.LowMachCombustionNSE;

In [1]:
string WFlowName = "UnstrainedDiffusionFlame";
WorkflowMgm.Init(WFlowName);

In [1]:
var myBatch = new MiniBatchProcessorClient(@"C:\deploy-Mini");
MiniBatchProcessor.Server.StartIfNotRunning();

In [1]:
string dbPath = @"C:\Databases\UnstrainedFlame_constantChemParams";
var myDb      = OpenOrCreateDatabase(dbPath);

In [1]:
Type solver = typeof(BoSSS.Application.LowMachCombustionNSE.LowMachCombustionNSEMain);

In [1]:
// ==========
// Parameters
// ==========
int[] dgDegree = new int[]{1,2,3,4};
int[] Resolutions = new int[]{3,4,5,6,7,8,9};
//int[] dgDegree = new int[]{2};
//int[] Resolutions = new int[]{5};

//int[] dgDegree = new int[]{3};
//int[] Resolutions = new int[]{4};
 
int[] Refinements = new int[]{1};
var  NofCells = Resolutions.ToList().Select(r=> (int)Math.Pow(2,r)).ToArray();
string baseJobName = "CounterDifFlame"   

In [1]:
NofCells

In [1]:
for (int dg = 0; dg < dgDegree.Length; dg++) {
    for (int r = 0; r < NofCells.Length; r++) {
        string ending        = "P" + dgDegree[dg] + "K" + NofCells[r];
        string jobName       = string.Format("FS_" + baseJobName + ending);
        string StartupString = string.Format("cs:BoSSS.Application.LowMachCombustionNSE.FullNSEControlExamples.FS_UnstrainedDiffusionFlame({0}, {1}, @\"{2}\")", dgDegree[dg], NofCells[r], dbPath);
        var oneJob           = new Job(jobName, solver);
        oneJob.SetControlStatement(StartupString);
        oneJob.NumberOfMPIProcs = NofCells[r] < 32? 1: 10;
        oneJob.NumberOfThreads = 1;
        oneJob.Activate(myBatch);
    }
}

In [1]:
WorkflowMgm.BlockUntilAllJobsTerminate();

In [1]:
for (int dg = 0; dg < dgDegree.Length; dg++) {
    for (int r = 0; r < NofCells.Length; r++) {
        string ending        = "P" + dgDegree[dg] + "K" + NofCells[r];
        string jobName       = string.Format("Full_" + baseJobName + ending);
        string StartupString = string.Format("cs:BoSSS.Application.LowMachCombustionNSE.FullNSEControlExamples.Full_UnstrainedDiffusionFlame({0}, {1}, @\"{2}\")", dgDegree[dg], NofCells[r], dbPath);
        var oneJob           = new Job(jobName, solver);
        oneJob.SetControlStatement(StartupString);
        oneJob.NumberOfMPIProcs = NofCells[r] < 32? 1: 10;
        oneJob.NumberOfThreads = 1;
        oneJob.Activate(myBatch);
    }
}

In [1]:
string dbPath = @"C:\Databases3\UnstrainedFlame";
var myDb      = OpenOrCreateDatabase(dbPath);

In [1]:
myDb.Sessions

In [1]:
// Now the flame sheet estimations should be deleted
WorkflowMgm.Sessions.Where(s => s.Name.StartsWith("FS_")).ForEach(x=>x.Delete(true));

In [1]:
WorkflowMgm.ResetSessionsCache();

In [1]:
a.KeysAndQueries

In [1]:
var a = myDb.Sessions.Pick(8);

In [1]:
var asd = (Convert.ToInt32(a.KeysAndQueries["Grid:NoOfCells"])) /3;

In [1]:
string dbPath = @"C:\Databases\UnstrainedFlame";
var myDb      = OpenOrCreateDatabase(dbPath);

In [1]:
string basepath = @"C:\Users\jfgj8\Dropbox\PaperdiffusionFlame\ama\data\UnstrainedFlame_deleteme\";
System.IO.Directory.CreateDirectory(basepath); // Create folder for storing data

In [1]:
string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure",
"MassFraction0","MassFraction1","MassFraction2","MassFraction3","MassFraction4" }; 

In [1]:
foreach (var varname in varnames) {
    foreach (int pDeg in dgDegree) {
       var pDegSessions = myDb.Sessions.Where(
            Si => (Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == pDeg 
        && (Convert.ToInt32(Si.KeysAndQueries["Grid:NoOfCells"]))/ 3 > 31
            ) // function which is true on all sessions we are interested in: 
         ).ToArray();
        Plot2Ddata pDegPlot =
         pDegSessions.ToEstimatedGridConvergenceData(varname,
           xAxis_Is_hOrDof: true, // false selects DOFs for x-axis 
           normType: NormType.L2_embedded );
            //Saving to a txt file
        pDegPlot.SaveTextFileToPublish(basepath+"\\"+  varname + "DG" + pDeg,false);
    }
}

In [1]:
myDb.Sessions.Pick(0).Export().Do()

In [1]:
// Plot error as "Value- Value_fine"
foreach (int pDeg in dgDegree) {
ISessionInfo[]  pDegSessions = WorkflowMgm.Sessions.Where(
        Si => (Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == pDeg)  // function which is true on all sessions we are interested in: 
     ).ToArray(); // Select databases with a given dgdegree
     
     var sortedSessions = pDegSessions.OrderBy(o => o.Timesteps[1].Grid.NumberOfCells).ToList();
     
      // Select timestep corresponding to the converged solution
     ITimestepInfo[] timesteps = sortedSessions.Select(s => s.Timesteps.Last()).ToArray();

     // pick the timestep with the finest mesh
     var fine    = timesteps.Last();
     var notfine = timesteps.Take(timesteps.Length-1);
    
     int c = 0;
     foreach(var L in notfine){
         DGField[] allErrors = new DGField[]{};
         foreach (var varname in varnames) {
            var fieldFine   = (fine.Fields.Where(s => (s.Identification == varname)).First()).CloneAs();
            var fieldCoarse = (L.Fields.Where(s => (s.Identification == varname)).First()).CloneAs(); //////////////////////
            DGField err     = fieldFine.CloneAs();
       err = DGField_Arithmetic.ScaledSummation(err,-1.0, fieldCoarse ,-1.0);
            fieldFine.Identification   = fieldFine.Identification  + "Fine"; 
            fieldCoarse.Identification = fieldCoarse.Identification  + "Coarse"; 
     
            err.Identification         = varname + "err";
       
            allErrors = ArrayTools.Cat<DGField>(allErrors, new DGField[] {err});
         }
              BoSSS.Solution.Tecplot.Tecplot.PlotFields(allErrors,"Errors_DG"+pDeg+ "_Res"+c  ,0,2);
          c++;    
     }
     
}

In [1]:
  BatchmodeConnector connector = new BatchmodeConnector();

In [1]:
   bool useCantera = true;

In [1]:
//get initial estimate from Cantera
//=======================================

if(useCantera){
connector.Cmd("diffflame"); // Call the cantera diffusion flame
connector.Cmd("fprintf('Number of points %d', length(z))");

connector.GetMatrix(null, "x");
connector.GetMatrix(null, "Temperature");
connector.GetMatrix(null, "MassFraction0");
connector.GetMatrix(null, "MassFraction1");
connector.GetMatrix(null, "MassFraction2");
connector.GetMatrix(null, "MassFraction3");
connector.GetMatrix(null, "VelocityX");
connector.Execute(false);
 }

In [1]:
Temperature

In [1]:
string[] varnames = new string[]{"VelocityX","VelocityY","Temperature", "Pressure"}; 

In [1]:
var ts = databases.Last().Sessions.Pick(0).Timesteps.Pick(0);//

In [1]:
// Coordinate array 
double xleft  = -1.0; 
double xright = 1.0;
int nCells    = 10;
double[]  _xNodes = GenericBlas.Linspace(xleft, xright, nCells + 1);
double[] Yvalues = new double[nCells+1];

In [1]:
foreach(var varname in varnames){
    var field = ts.Fields.Where( Fi => (Fi.Identification == varname)).Last();
        for(int n = 0 ; n < _xNodes.Length; n++){
            Yvalues[n] = field.ProbeAt(new double[]{ _xNodes[n], 0.0});    // Pick value along the central axis. and Store it in array
    }
        
}

In [1]:
string bpath          = @"C:\tmp\ConvPlots\data\CanteraComparison";
string pathWithoutExt = System.IO.Path.ChangeExtension(bpath, null);
string newPath        = pathWithoutExt + "Data.txt";

In [1]:
using (StreamWriter s = new StreamWriter(newPath)) {
    s.WriteLine("x\ty");
    foreach (var varname in varnames) {
        for (int i = 0; i < group.Abscissas.Length; i++) {
            s.Write(_xNodes[i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
            s.WriteLine();
        }
    }
}